## Earth Engine and Live Tiles for Web Maps

### 1. Build a tile export service mixing precalculated tiles with live tiles from Earth Engine

This Section shows an example of how to mix pre-calculated tiles from Earth Engine (hosted on GCS) with live-calculated tiles from Earth Engine (past zoom level 12).

Based on code snippets drawn from [here](https://github.com/wri/gfw-api/blob/master/gfw/gee_tiles.py).

Note: the tokens for the live Earth Engine tiles expire every 3 days, and so this code will need to be re-run with an authorised EE account to generate valid (live) tiles.

### 2. Best Sentinel 2 imagery for specific points and times

This section shows how to select the best (cloud free) images of Sentinel 2, for specific times and locations, and present them on a web map.

In [ ]:
import os
import ee
import json

ee.Initialize()

In [ ]:
def tile_url(image, viz_params=None):
    """Create a target url for tiles for an image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (base_url + '/map/' + d['mapid'] + '/{z}/{x}/{y}?token=' + d['token'])
    return url

### EE pansharpened image

Below calculates a pan-sharpended image from Landsat, and creates a tile url with temporary authentitication.

In [ ]:
collection = ee.ImageCollection('LANDSAT/LC8_L1T').filterDate('2016-01-01T00:00','2017-01-01T00:00')

composite = ee.Algorithms.SimpleLandsatComposite(collection=collection, percentile=50,
                                                 maxDepth=80, cloudScoreRange=1, asFloat=True)

hsv2 = composite.select(['B4', 'B3', 'B2']).rgbToHsv()

sharpened2 = ee.Image.cat([hsv2.select('hue'), hsv2.select('saturation'),
                           composite.select('B8')]).hsvToRgb().visualize(gain=1000, gamma= [1.15, 1.4, 1.15])


ee_tiles = tile_url(sharpened2)
ee_tiles

## Use the vizzuality microservice to retrieve the url

Microservice based on the preceeding code should give back the tileset

In [ ]:
import requests

In [ ]:
r = requests.get('https://staging-api.globalforestwatch.org/v1/landsat-tiles/2015')
print(r.status_code)
r.json()

In [ ]:
ee_tiles= r.json().get('data').get('attributes').get('url')
print(ee_tiles)

### Display the results on a map

In [ ]:
pre_calculated_tileset="https://storage.googleapis.com/landsat-cache/2015/{z}/{x}/{y}.png"

In [ ]:
import folium

In [ ]:
map = folium.Map(location=[28.29, -16.6], zoom_start=2, tiles='Mapbox Bright' )

In [ ]:
map.add_tile_layer(tiles=pre_calculated_tileset, max_zoom=11, min_zoom=0, attr='Earth Engine tiles by Vizzuality')
map.add_tile_layer(tiles=ee_tiles, max_zoom=20, min_zoom=13, attr="Live EE tiles")

In [ ]:
map

## 2. Sentinel

In [1]:
import folium
import os
import ee
import json
ee.Initialize()

The below Earth Engine code returns the optimum cloud-free image for a given point and time range.

In [8]:
def tile_url(image, viz_params=None):
    """Create a target url for tiles for an image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (base_url + '/map/' + d['mapid'] + '/{z}/{x}/{y}?token=' + d['token'])
    return url


def proxy_sentinel(lat, lon, start, end):
    """
    Sentinel covers all continental land surfaces (including inland waters) between latitudes 56° south and 83° north
        all coastal waters up to 20 km from the shore
        all islands greater than 100 km2
        all EU islands
        the Mediterranean Sea
        all closed seas (e.g. Caspian Sea).
    
    Filter by tiles that intersect a lat,lon point, and are within a date range, and have less than
    10% cloud cover, then find the lowest scoring cloud image.
    
    Note, we first filter by cloud less than X% and then pick the top, rather than just directly pick
    the best cloud scoring image, as these operations default to a subset of images. So we want to 
    pick the best image, from a pre-selected subset of good images.
    
    Note the url generated expires after a few days and needs to be refreshed.
    e.g. variables
    lat = -16.66
    lon = 28.24
    start = '2017-01-01'
    end = '2017-03-01'
    """
    #if lat >= 83 or lat <= -56:
    #    return {'status': 'Latitute {0} invalid: Must be between -56 and 83'.format(lat)}
    #else:
    try:
        point = ee.Geometry.Point(lat, lon)
        S2 = ee.ImageCollection('COPERNICUS/S2'
                               ).filterDate(
                                start, end).filterBounds(
                                point).sort('CLOUDY_PIXEL_PERCENTAGE', True).first()
        S2 = ee.Image(S2)
        d = S2.getInfo()       # grab a dictionary of the image metadata
        S2 = S2.divide(10000)  # Convert to Top of the atmosphere reflectance
        S2 = S2.visualize(bands=["B4", "B3", "B2"], min=0, max=0.3, opacity=1.0) # Convert to styled RGB image
        image_tiles = tile_url(S2)
        boundary = ee.Feature(ee.Geometry.LinearRing(d.get('properties').get("system:footprint").get('coordinates')))
        boundary_tiles = tile_url(boundary, {'color': '4eff32'})
        meta = get_image_metadata(d)
        output = {}
        output = {'boundary_tiles': boundary_tiles,
                  'image_tiles': image_tiles,
                  'metadata': meta,
                  'sucsess': True}

        return output
    except:
        return {'sucsess': False}


def get_image_metadata(d):
    """Return a dictionary of metadata"""
    image_name = d.get('id')
    date_info = image_name.split('COPERNICUS/S2/')[1]
    date_time = ''.join([date_info[0:4],'-',date_info[4:6],'-',date_info[6:8],' ',
                      date_info[9:11],':',date_info[11:13],':',date_info[13:15],"Z"])
    product_id = d.get('properties').get('PRODUCT_ID')
    meta = {}
    meta = {'image_name': image_name, 'date_time': date_time, 'product_id': product_id}
    return meta

Sentinel-2: MultiSpectral Instrument (MSI), Level-1C
Jun 23, 2015 - Aug 30, 2017

Info on [Sentinel Naming conventions](https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention)

First comes YYYYMMDDHHMMSS then _ then 15 characters of ID and then _ then tile number info

In [9]:
%%time
# london
#lon = 51.64167220085054
#lat = 0.03

# oslo
#lon = 10.745
#lat = 59.922

# Tenerife
lat = -16.644
lon = 28.266

# Fiji
#lat = 177.825
#lon = -17.916

start ='2017-01-01'
end ='2017-01-10'

sentinel = proxy_sentinel(lon=lon, lat=lat, start=start, end=end)



CPU times: user 29.4 ms, sys: 6.16 ms, total: 35.6 ms
Wall time: 35.2 s


In [10]:
if sentinel.get('sucsess'):
    pass
else:
    raise ValueError('no data')

print(sentinel.get('metadata'))

sentinel_map = folium.Map(location=[lon, lat], zoom_start=9, tiles='Mapbox Bright' )
sentinel_map.add_tile_layer(tiles=sentinel.get('image_tiles'), max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map.add_tile_layer(tiles=sentinel.get('boundary_tiles'), max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map

{'image_name': 'COPERNICUS/S2/20170101T115212_20170101T115212_T28RCS', 'date_time': '2017-01-01 11:52:12Z', 'product_id': 'S2A_MSIL1C_20170101T115212_N0204_R123_T28RCS_20170101T115212'}


In [ ]:
sentinel

## Sentinel (via the microservice)

We have hooked-up a version of the above sentinel code to our API ([here](https://github.com/gfw-api/gfw-analysis-gee)). An example of how to call it to get the same results as above is as follows: 

{'lat':'-16.644','lon':'28.266', 'start':'2017-01-01', 'end': "2017-09-10"} Wall time: 25.6 s
{'lat':'-16.644','lon':'28.266', 'start':'2017-01-01', 'end': "2017-01-10"} Wall time: 25.8 s


In [ ]:
import requests
import folium

In [ ]:
%%time
url = "https://staging-api.globalforestwatch.org/v1/sentinel-tiles"
params= {'lat':'-16.644','lon':'28.266', 'start':'2016-01-01', 'end': "2017-01-10"}
r = requests.get(url, params=params)
r.status_code

In [ ]:

print(r.json())

dt = r.json().get('data').get('attributes').get('date_time')
boundary = r.json().get('data').get('attributes').get('url_boundary')
sentinel_image = r.json().get('data').get('attributes').get('url_image')

```
{'data': {'attributes': {'date_time': '2017-01-01 11:52:12Z', 'product_id': 'S2A_MSIL1C_20170101T115212_N0204_R123_T28RCS_20170101T115212', 'url_boundary': 'https://earthengine.googleapis.com/map/4b1b9c6f82d50796562521502bc4d9a2/{z}/{x}/{y}?token=fb2181f663a8f2895224fecddd8b1ec4', 'url_image': 'https://earthengine.googleapis.com/map/0341ef17ae8e75cf0b53b5de5fd767ff/{z}/{x}/{y}?token=df18de8c707463d20912e8699f7801b8'}, 'id': None, 'type': 'sentinel_tiles_url'}}

```

In [ ]:
sentinel_map = folium.Map(location=[float(params['lon']), float(params['lat'])], zoom_start=9, tiles='Mapbox Bright' )
sentinel_map.add_tile_layer(tiles=sentinel_image, max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map.add_tile_layer(tiles=boundary, max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map